In [21]:
import os
import sys

from dotenv import dotenv_values
from web3 import Web3
import requests
import json
from pprint import pprint

In [22]:
conf = dotenv_values(dotenv_path='../.env')
web3 = Web3(Web3.HTTPProvider(f"https://mainnet.infura.io/v3/{conf['infura']}"))

In [23]:
# helper functions for ABIs and reading what contract contains

def get_abi(address):
    r = requests.get(
                ("https://api.etherscan.io/api"
                    "?module=contract"
                    "&action=getabi"
                    f"&address={address}"
                    f"&apikey={conf['etherscan']}")
                    )
        
    return r.json()['result']

def contract_functions(contract):
    print(*contract.functions, sep='\n')

In [24]:
# start from fuse pool directory and implementation

directory_address = '0x835482FE0532f169024d5E9410199369aAD5C77E'
directory_implementation = '0xd662efb05e8cafe35d1558b8b5323c73e2919abd'
directory_contract = web3.eth.contract(directory_address, abi = get_abi(directory_implementation))

contract_functions(directory_contract)

_editAdminWhitelist
_editDeployerWhitelist
_setDeployerWhitelistEnforcement
adminWhitelist
bookmarkPool
deployPool
deployerWhitelist
enforceDeployerWhitelist
getAllPools
getBookmarks
getPoolsByAccount
getPublicPools
getPublicPoolsByVerification
initialize
owner
poolExists
pools
renounceOwnership
setPoolName
transferOwnership


In [25]:
# Directory contains list of all created pools, choose the one with most assets

pools = directory_contract.functions.getAllPools().call()
pool = pools[6]
pool

("Tetranode's Pool",
 '0x4702D39c499236A43654c54783c3f24830E247dC',
 '0x814b02C1ebc9164972D888495927fe1697F0Fb4c',
 12149219,
 1617221487)

In [26]:
# create the pool contract object

pool_address = pool[2]
pool_contract = web3.eth.contract(pool_address, abi=get_abi(pool_address))
contract_functions(pool_contract)

pool_implementation= pool_contract.functions.comptrollerImplementation().call()
print(pool_implementation)
pool_true = web3.eth.contract(pool_address, abi=get_abi(pool_implementation))

# find the markets that the pool contains
markets = pool_true.functions.getAllMarkets().call()
markets

_acceptAdmin
_acceptImplementation
_renounceAdminRights
_renounceFuseAdminRights
_setPendingAdmin
_setPendingImplementation
admin
adminHasRights
comptrollerImplementation
fuseAdminHasRights
pendingAdmin
pendingComptrollerImplementation
0x94B2200d28932679DEF4A7d08596A229553a994E


['0x989273ec41274C4227bCB878C2c26fdd3afbE70d',
 '0x558a7A68C574D83f327E7008c63A86613Ea48B4f',
 '0x325e3257286EB040AABB1128c21a55cF82e25901',
 '0x2130528060141222F0614ff80A756A3B2A24fE59',
 '0xAD1716680024F6F9AEA57Ad28b8C4Ecd2f5670CC',
 '0xdb55B77F5E8a1a41931684Cf9e4881D24E6b6CC9',
 '0x8691927a91A032c23B895130074669f52CF6b1e7',
 '0x59Bd6774C22486D9F4FAb2D448dCe4F892a9Ae25',
 '0xF6551C22276b9Bf62FaD09f6bD6Cad0264b89789',
 '0xcA56Af76B656212d768842246bF4893b56C02ABc',
 '0x1531C1a63A169aC75A2dAAe399080745fa51dE44',
 '0x185Ab80A77D362447415a5B347D7CD86ecaCC87C',
 '0xf65155C9595F99BFC193CaFF0AAb6e2a98cf68aE',
 '0xf9f0EFFE60F56e6846505501903AD047B8011C3e',
 '0xC12b58D31b97DBd7F092db5cc69ad321a0AD747E',
 '0xF317379B10D370Fec6B8103Ef2da5007d1890def',
 '0xE33928B720799127A052B65498b322A206351441',
 '0xeb37cE0DB663A742Df93E23EA7ba78016e82BE39']

In [27]:
# choose dai market and create the contract object

dai = markets[0]
dai_contract = web3.eth.contract(dai, abi=get_abi(dai))

dai_implementation = dai_contract.functions.implementation().call()
dai_true = web3.eth.contract(dai, abi=get_abi(dai_implementation))

contract_functions(dai_true)

_acceptAdmin
_addReserves
_becomeImplementation
_reduceReserves
_renounceAdminRights
_renounceFuseAdminRights
_resignImplementation
_setAdminFee
_setComptroller
_setFuseFee
_setInterestRateModel
_setPendingAdmin
_setReserveFactor
_withdrawAdminFees
_withdrawFuseFees
accrualBlockNumber
accrueInterest
admin
adminFeeMantissa
adminHasRights
allowance
approve
balanceOf
balanceOfUnderlying
borrow
borrowBalanceCurrent
borrowBalanceStored
borrowIndex
borrowRatePerBlock
comptroller
decimals
exchangeRateCurrent
exchangeRateStored
fuseAdminHasRights
fuseFeeMantissa
getAccountSnapshot
getCash
implementation
initialize
initialize
interestRateModel
isCEther
isCToken
liquidateBorrow
mint
name
pendingAdmin
redeem
redeemUnderlying
repayBorrow
repayBorrowBehalf
reserveFactorMantissa
seize
supplyRatePerBlock
symbol
totalAdminFees
totalBorrows
totalBorrowsCurrent
totalFuseFees
totalReserves
totalSupply
transfer
transferFrom
underlying


In [32]:
# each market also has information and supplied and borrowed assets
supply = dai_true.functions.totalSupply().call()
borrows = dai_true.functions.totalBorrows().call()

# this information is fed to interest rate model to get supply/borrowRatePerBlock
supply_per_block = dai_true.functions.supplyRatePerBlock().call()
borrow_per_block = dai_true.functions.borrowRatePerBlock().call()

# supply and borrow rate per block are converted into annualized return by formula given in docs:
# https://docs.rari.capital/fuse/#calculating-the-apy-using-rate-per-block
eth_mant = 1 * 10 ** 18
days_per_year = 365
blocks_per_day = 6500

def rate_to_apy(pool_rate):
    # supply/borrow_apy = ((((supply_per_block / eth_mant) * blocks_per_day + 1) ** day_per_year) - 1) * 100
    decimal_conversion = pool_rate / eth_mant
    per_day = decimal_conversion * blocks_per_day + 1
    annualized = per_day ** days_per_year
    percentage = (annualized - 1) * 100
    return percentage

print(borrows, supply)
print(supply_per_block, borrow_per_block)
print(rate_to_apy(supply_per_block), rate_to_apy(borrow_per_block))

# Numbers match that of the UI

25858710540976168457875505 37676543849924005103593260
54016560261 100584182779
13.670280628648591 26.941732739493606


In [ ]:
# to model the capital that can be deployed, we need to see how supply/demandRatePerBlock are created
# Implementation for the function:
# 

# interestRateModel.getSupplyRate(getCashPrior(), totalBorrows, totalReserves + totalFuseFees + totalAdminFees, reserveFactorMantissa + fuseFeeMantissa + adminFeeMantissa);
# interestRateModel.getBorrowRate(getCashPrior(), totalBorrows, totalReserves + totalFuseFees + totalAdminFees);

# therefore both functions are found 

To model the capital that can be deployed, we need to see how supply/demandRatePerBlock are created.
Implementation is found here:

```https://etherscan.io/address/0x67e70eeb9dd170f7b4a9ef620720c9069d5e706c#code```

And the important functions are:

```interestRateModel.getSupplyRate();``` which has inputs: 
```getCashPrior(), totalBorrow, totalReserves + totalFuseFees + totalAdminFees, reserveFactorMantissa + fuseFeeMantissa + adminFeeMantissa```


- ```getCashPrior``` getCash()

- ```totalBorrow``` totalBorrowsCurrent()

- ```totalReserves``` totalReserves()

- ```totalFuseFees``` totalFuseFees()

- ```totalAdminFees``` totalAdminFees()

- ```reserveFactorMantissa``` reserveFactorMantissa()

- ```fuseFeeMantissa``` fuseFeeMantissa()

- ```adminFeeMantissa``` adminFeeMantissa()

and 
```interestRateModel.getBorrowRate();```
Inputs:
```getCashPrior(), totalBorrows, totalReserves + totalFuseFees + totalAdminFees```

In [45]:
# test by pulling those values above and calling the getSupplyRate function from interest rate model
# Interest rate model is found in the market contract, DAI:
# https://etherscan.io/address/0xb579d2761470bba14018959d6dffcc681c09c04b#readContract

# get cash returns balance of underlying (eg. DAI) in the contract
# getCash = underlying.balanceOf(address.this)
get_cash_prior = dai_true.functions.getCash().call()
total_borrows_current = dai_true.functions.totalBorrowsCurrent().call()
total_reserves = dai_true.functions.totalReserves().call()
total_fuse_fees = dai_true.functions.totalFuseFees().call()
total_admin_fees = dai_true.functions.totalAdminFees().call()
reserve_factor_mant = dai_true.functions.reserveFactorMantissa().call()
fuse_fee_mant = dai_true.functions.fuseFeeMantissa().call()
admin_fee_mant = dai_true.functions.adminFeeMantissa().call()

print(get_cash_prior, total_borrows_current, total_reserves, total_fuse_fees, total_admin_fees, reserve_factor_mant, fuse_fee_mant, admin_fee_mant)
print()

interest_rate_address = dai_true.functions.interestRateModel().call()
print(interest_rate_address)
print()
interest_rate_contract = web3.eth.contract(interest_rate_address, abi = get_abi(interest_rate_address))
#contract_functions(interest_rate_contract)
print()

calculated_rate_supply = interest_rate_contract.functions.getSupplyRate(
    get_cash_prior,
    total_borrows_current,
    total_reserves + total_fuse_fees + total_admin_fees,
    reserve_factor_mant + fuse_fee_mant + admin_fee_mant
).call()

# compare calculated rate with queried rate
queried_rate_supply = dai_true.functions.supplyRatePerBlock().call()

print(f"True: {queried_rate_supply}\nCalculated: {calculated_rate_supply}")
print(f"True: {rate_to_apy(queried_rate_supply)}\nCalculated: {rate_to_apy(calculated_rate_supply)}")
print()

calculated_rate_borrow = interest_rate_contract.functions.getBorrowRate(
    get_cash_prior,
    total_borrows_current,
    total_reserves + total_fuse_fees + total_admin_fees
).call()

# compare calculated rate with queried rate
queried_rate_borrow = dai_true.functions.borrowRatePerBlock().call()

print(f"True: {queried_rate_borrow}\nCalculated: {calculated_rate_borrow}")
print(f"True: {rate_to_apy(queried_rate_borrow)}\nCalculated: {rate_to_apy(calculated_rate_borrow)}")



17604902927838865172033136 25861169468512115190837609 27737236040679707770412 101236514769169626053942 0 0 100000000000000000 0

0xb579D2761470BBa14018959d6dfFcc681c09c04b


True: 54024281501
Calculated: 54024425835
True: 13.672362201977627
Calculated: 13.672401113416498

True: 100590919763
Calculated: 100591045693
True: 26.9437604024644
Calculated: 26.943798304539392


In [54]:
#contract_functions(dai_true)
underlying = dai_true.functions.underlying().call()
underlying_contract = web3.eth.contract(address=underlying, abi=get_abi(underlying))
print(underlying_contract.functions.balanceOf(dai_true.address).call())

dai_true.functions.getCash().call()

16534006826133802757727581


16534006826133802757727581

In [ ]:
# each market has their own interest rate model which is used to calculate the borrow and supply rates

interest_rate_address = dai_true.functions.interestRateModel().call()
print(interest_rate_address, '\n')
interest_rate_contract = web3.eth.contract(interest_rate_address, abi = get_abi(interest_rate_address))
contract_functions(interest_rate_contract)